# Segmenting and clustering Toronto Neighborhood

Libraries first 

In [1]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup as bs
import requests
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

## some more libraries as used in exercise
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  53.72 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.04 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.24 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  47.53 MB/s
Libraries imported.


Use Beautiful soap to extract data from wikipedia

In [61]:
raw_wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
extract = bs(raw_wiki)
print(extract.prettify())    #Not printing it to keep the notebook short. It works.

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/envs/DSX-Python35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


We need only data that constitute table. So lets target the table content. 

In [62]:
table = extract.find('table',{'class':'wikitable sortable'})
table
## Not displaying extracts to keep the notebook length short. It works. 

Now we have html markers'tr' and 'td' to identify the fields. 

1.Use Findall to find the text

2.Append to append the text based on conditions

3.Populate list by iterating through the loop 

!!Remember to remove "Not Assigned"


In [4]:
postcode = []
borough = []
neighborhood = []

for row in table.find_all('tr')[1:]:  # Find tr . Leave first row for heading
    borough_list = row.find_all('td')[1] # Get boroughs.  second column
    if (borough_list.text.strip() != 'Not assigned'):  # avoid 'Not assigned'
        borough.append(borough_list.text.strip())    # Append to list borough[]
        postcode_list = row.find_all('td')[0]  #  Find td.  first column
        postcode.append(postcode_list.text.strip()) # Append to list postcode[]
        neighborhood_list = row.find_all('td')[2] # Remaining td . third column
        if (neighborhood_list.text.strip() == 'Not assigned'): 
            neighborhood.append(borough_list.text.strip()) 
        else:
            neighborhood.append(neighborhood_list.text.strip())  #Append to list neighborhood[]


In [5]:
df = pd.DataFrame()
## populate data frame with 3 colums we made.
df['PostalCode'] = postcode
df['Borough'] = borough
df['Neighborhood'] = neighborhood
## df contains repeated values. We use groupby and join

df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
df.shape


(103, 3)

To get longitudes and latitudes for items in our dataframe

In [7]:
## CSV file provided has list of postcode latitude longitudes. Easier than geocoder method. 
coordinate_df = pd.read_csv('http://cocl.us/Geospatial_data')
coordinate_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df.columns  ## check the column indices

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

Merge the co ordinate data from dataframe coordinate_df to our data frame df

In [8]:
## take each postcode in df and iterate through coordinate_df to find match
## we can also do inner join method. but I prefer loops - easy to understand
for column in df.columns[:1]:
    for column1 in coordinate_df.columns[:1]:
        if df[column][1] == coordinate_df[column1][1]:
            df['Latitude'] = coordinate_df["Latitude"]
            df['Longitude'] = coordinate_df["Longitude"]
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Now the task is //Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto //

In [9]:
df_toronto = df[df.Borough.str.contains('Toronto')].reset_index(drop=True)
## df_toronto.shape
df_toronto.head(38)  ## there are 38 Boroughs with Toronto and unique postcode

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


Create Folium map

In [10]:
### We can find lat, long of Toronto from google map.
### Better to find the average of lat long from our dataset
lat_toronto = df_toronto['Latitude'].mean()
long_toronto = df_toronto['Longitude'].mean()
print('Toronto lat {}, Toronto long {}'.format(lat_toronto, long_toronto))

Toronto lat 43.66726218421052, Toronto long -79.38988323421053


In [11]:
# Create map of Toronto centered around the average lat, long
map_toronto = folium.Map(location=[lat_toronto , long_toronto], zoom_start=12)


# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='B',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto



Let us replicate the workflow in Newyork neighborhood example to segment and cluster the neighborhood.

Use Foursqure API to explore

In [14]:
CLIENT_ID = 'MXE3UWLG2E3VX3S1LEBYEA1EE3EIJTLK0BBASJDJ414X3ASW' # your Foursquare ID
CLIENT_SECRET = 'YY1YH5US5V4WWQEL5GAMSW1VYZDNACUXRCEBWL23JQDL5OIS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MXE3UWLG2E3VX3S1LEBYEA1EE3EIJTLK0BBASJDJ414X3ASW
CLIENT_SECRET:YY1YH5US5V4WWQEL5GAMSW1VYZDNACUXRCEBWL23JQDL5OIS


Find the co ordinates corresponding top 100 venues downtown Toronto around University of Toronto

In [13]:
### It is easier to get lat long from table above. But we use the method used in exercise.
### Its location is 25th index in our Toronto table. 
df_toronto.loc[25, 'Neighborhood']

neighborhood_latitude = df_toronto.loc[25, 'Latitude'] 
neighborhood_longitude = df_toronto.loc[25, 'Longitude'] 
neighborhood_name = df_toronto.loc[25, 'Neighborhood'] 

print('Latitude and Longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and Longitude values of Harbord, University of Toronto are 43.6626956, -79.4000493.


Lest get top 100 venues around 500m radius

In [15]:
### create a url
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=MXE3UWLG2E3VX3S1LEBYEA1EE3EIJTLK0BBASJDJ414X3ASW&client_secret=YY1YH5US5V4WWQEL5GAMSW1VYZDNACUXRCEBWL23JQDL5OIS&v=20180605&ll=43.6626956,-79.4000493&radius=500&limit=100'

In [63]:
results = requests.get(url).json()
results

From the Foursquare lab in the previous module, we know that all the information is in the items key. 
Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into datframe

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Rasa,Restaurant,43.662757,-79.403988
2,Almond Butterfly,Bakery,43.662836,-79.403365
3,Piano Piano,Italian Restaurant,43.662949,-79.402898
4,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746


In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
## How many venues returned?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

34 venues were returned by Foursquare.


Repeat the process for all neighborhood in the table

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [25]:
### Get venues for each neighborhood using above function
borough_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [28]:
borough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [27]:
borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,20,20,20,20,20,20
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,81,81,81,81,81,81
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [29]:
### How many unique categories?
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 234 uniques categories.


In [39]:
# one hot encoding
Toronto_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
Toronto_onehot.shape

(1693, 234)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head(10)

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.0,...,0.0,0.0,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.050000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.0,...,0.0,0.0,0.012346,0.000000,0.000000,0.012346,0.000000,0.000000,0.00,0.012346
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.050000,0.000000,0.050000,0.010000,0.000000,0.000000,0.00,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.0,...,0.0,0.0,0.000000,0.011765,0.011765,0.000000,0.011765,0.011765,0.00,0.011765


In [33]:
Toronto_grouped.shape

(38, 234)

Get top 10 venues for each neighborhood

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Burger Joint,Gym,Asian Restaurant,American Restaurant,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Café,Bakery,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Climbing Gym,Falafel Restaurant,Convenience Store,Burrito Place,Stadium,Caribbean Restaurant,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Park,Brewery,Skate Park,Burrito Place,Butcher,Restaurant,Recording Studio,Comic Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Plane,Harbor / Marina
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pub,Pizza Place,Italian Restaurant,Bakery,Diner,Indian Restaurant,Chinese Restaurant
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Bubble Tea Shop,Salad Place,Sushi Restaurant,Ice Cream Shop,Thai Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Chinese Restaurant,Mexican Restaurant,Caribbean Restaurant,Gaming Cafe
8,Christie,Grocery Store,Café,Park,Diner,Convenience Store,Baby Store,Italian Restaurant,Coffee Shop,Athletics & Sports,Nightclub
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Burger Joint,Dance Studio,Gastropub,Mediterranean Restaurant,Men's Store


Now Comes K means to cluster into 5 clusters

In [48]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [50]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged




,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Liquor Store,Sandwich Place,Italian Restaurant,Steakhouse,Brewery,Movie Theater,Ice Cream Shop,Pub,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Park,Brewery,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Burger Joint,Dog Run,Food & Drink Shop,Clothing Store,Sandwich Place,Park,Breakfast Spot,Gym,Electronics Store
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Gift Shop,Ice Cream Shop,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Café,Restaurant,Sushi Restaurant,Coffee Shop,Pharmacy,Seafood Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,4,Playground,Restaurant,Tennis Court,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Pub,Coffee Shop,Pizza Place,American Restaurant,Supermarket,Fried Chicken Joint,Sandwich Place,Sports Bar,Sushi Restaurant,Bagel Shop


create Map

In [55]:
# create map
map_clusters = folium.Map(location=[lat_toronto , long_toronto], zoom_start=12)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters 

In [56]:
## cluster 1
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Coffee Shop,Pub,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,East Toronto,0,Park,Liquor Store,Sandwich Place,Italian Restaurant,Steakhouse,Brewery,Movie Theater,Ice Cream Shop,Pub,Fish & Chips Shop
3,East Toronto,0,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Park,Brewery,Seafood Restaurant
5,Central Toronto,0,Hotel,Burger Joint,Dog Run,Food & Drink Shop,Clothing Store,Sandwich Place,Park,Breakfast Spot,Gym,Electronics Store
6,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Gift Shop,Ice Cream Shop,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop
7,Central Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Café,Restaurant,Sushi Restaurant,Coffee Shop,Pharmacy,Seafood Restaurant
9,Central Toronto,0,Pub,Coffee Shop,Pizza Place,American Restaurant,Supermarket,Fried Chicken Joint,Sandwich Place,Sports Bar,Sushi Restaurant,Bagel Shop
11,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Pub,Pizza Place,Italian Restaurant,Bakery,Diner,Indian Restaurant,Chinese Restaurant
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Burger Joint,Dance Studio,Gastropub,Mediterranean Restaurant,Men's Store


In [57]:
## cluster 2
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [58]:
## cluster 3
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [59]:
## cluster 4
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
23,Central Toronto,3,Park,Trail,Sushi Restaurant,Jewelry Store,Bus Line,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Electronics Store


In [60]:
## cluster 5
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4,Playground,Restaurant,Tennis Court,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [ ]:
### The END ####